# Histopathologic Cancer Detection Re-run

With this notebook i am re-running the model for 25 more epochs.

# Imports

In [ ]:
import numpy as np
import pandas as pd 
import random
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Conv2D, MaxPool2D
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import *

from keras_preprocessing.image import ImageDataGenerator

# Parameters

In [ ]:
BATCH_SIZE = 64
IMG_SIZE = 96
RANDOM_SEED = 1982

# Dataset

In [ ]:
dataset = '/kaggle/input/histopathologic-cancer-detection/'
train_path = dataset+'train/'
test_path = dataset+'test/'

In [ ]:
data = pd.read_csv(dataset+'train_labels.csv', dtype=str)
print('Training Set Size:', data.shape)
data['path'] = data.id + '.tif'
data.head()

# Test Train split by 70/30

In [ ]:
train, valid = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data.label)

print(train.shape)
print(valid.shape)

# Data Generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_loader = train_datagen.flow_from_dataframe(
    dataframe = train,
    directory = train_path,
    x_col = 'path',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = RANDOM_SEED,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (IMG_SIZE,IMG_SIZE)
)

valid_loader = train_datagen.flow_from_dataframe(
    dataframe = valid,
    directory = train_path,
    x_col = 'path',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = RANDOM_SEED,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (IMG_SIZE,IMG_SIZE)
)

# CNN Model using ResNet50

In [ ]:
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

cnn = tf.keras.models.load_model('../input/cancerdetection-tl-v02/cancer_model_v01.h5')

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(96,96,3),
                                         include_top=False,
                                         weights='imagenet')

base_model.trainable = True
#K.set_value(cnn.optimizer.learning_rate, 0.0001)

In [ ]:
opt = tf.keras.optimizers.Adam(0.0001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])
cnn.summary()

In [ ]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

In [ ]:
%%time 

# Complete one or more training runs. 
# Display training curves after each run. 

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 20,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1,
    use_multiprocessing=True, 
    workers=8
)

In [ ]:
pickle_file = open("../input/cancerdetection-tl-v02/cancer_history_v00.pkl", "rb")
history = pickle.load(pickle_file)
pickle_file.close()

In [ ]:
for k in h2.history.keys():
    history[k] += h2.history[k]
    
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
cnn.save('LK_HCD_CNN_TLV2_Model.h5')
pickle.dump(history, open(f'LP_HCD_CNN_Model_V2_History.pkl', 'wb'))